## FISH SEMANTIC SEGMENTATION

In this notebook we investigate the segmentation of fish

In [ ]:
# %matplotlib notebook
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import random
import glob
import numpy as np
# import skimage.io as io
import cv2
import json
from pycocotools.coco import COCO
import mcoco.coco as coco

import keras
random.seed(148)

Create data generators

In [ ]:
train_coco_files = glob.glob("/root/data/aquabyte-images-crops/segmentation-trainer/Train/*")
test_coco_files = glob.glob("/root/data/aquabyte-images-crops/segmentation-trainer/Test/*")
print("Number of training files: {}".format(len(train_coco_files)))
print("Number of test files: {}".format(len(test_coco_files)))

In [ ]:
train_coco_files

In [ ]:
dataset_train = coco.CocoDataset()
for coco_file in train_coco_files:
    dataset_train.load_coco(coco_file, class_ids=[1])
dataset_train.prepare()
print("Number of train images: {}".format(dataset_train.num_images))

In [ ]:
dataset_validate = coco.CocoDataset()
for coco_file in test_coco_files:
    print(coco_file)
    dataset_validate.load_coco(coco_file, class_ids=[1])
dataset_validate.prepare()
print("Number of val images: {}".format(dataset_validate.num_images))

In [ ]:
input_shape = (768, 256)
batch_size = 8

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, dataset, batch_size, input_shape):
        'Initialization'
        self.dataset = dataset
        self.image_ids = dataset.image_ids
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.image_ids) // self.batch_size

    def __getitem__(self, index):
        'Generate one batch of data'
        xbatch = []
        ybatch = []
        start = index*self.batch_size
        for i in range(self.batch_size):
            image_id = self.image_ids[start+i]
            image = self.dataset.load_image(image_id)
            image = cv2.resize(image, self.input_shape)
            
            mask = self.dataset.load_mask(image_id)[0]
            mask = np.array(mask, dtype=np.uint8)
            mask = mask.squeeze()
            mask = cv2.resize(mask, self.input_shape)
            mask = np.expand_dims(mask, axis=-1)
            xbatch.append(image)
            ybatch.append(mask)
        xbatch = np.array(xbatch)
        ybatch = np.array(ybatch)
        return xbatch, ybatch
            
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        np.random.shuffle(self.image_ids)

In [ ]:
len(dataset_train.load_mask(0))

In [ ]:
traingen = DataGenerator(dataset_train, batch_size, input_shape)
valgen = DataGenerator(dataset_validate, batch_size, input_shape)

In [ ]:
# %matplotlib notebook

In [ ]:
x, y = valgen[0]
for i in range(batch_size):
    plt.imshow(x[i, ...])
    plt.imshow(y[i, ...].squeeze(), alpha=0.5)
    plt.show();

Create model

In [ ]:
from unet import get_unet, jaccard_coef_loss, jaccard_coef_int
import os
from keras.optimizers import Adam

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
model = get_unet(3, 256, 768, classes=1)

In [ ]:
lr=0.001
adam = Adam(lr=lr)
model.compile(adam, loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])

Train

In [ ]:
# start training# start 
history = model.fit_generator(generator=traingen,
                              epochs=50,
                              verbose=1,
                              validation_data=valgen)


In [ ]:
h = history.history

In [ ]:
%matplotlib inline
%matplotlib notebook

In [ ]:
plt.plot(h["loss"])
plt.plot(h["val_loss"])
plt.ylim([0, 2])
plt.show()

In [ ]:
plt.plot(h['val_jaccard_coef_int'])
plt.show()

In [ ]:
# model.save("/root/data/models/single_fish_unet_segmentation.h5")

# looking at predictions

In [ ]:
input_shape

In [ ]:
image = cv2.imread("/root/data/lice-data/crops/blom-kjeppevikholmen/2019-03-30/2/left_blom-kjeppevikholmen_2_1553953407251_0_1390_3642_2847.jpg")

In [ ]:
image = cv2.resize(image, input_shape)

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(image)
plt.show()

In [ ]:
ypred = model.predict_on_batch(np.expand_dims(image, axis=0))

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(image)
plt.imshow(ypred.squeeze(), alpha=0.5)
plt.show()

In [ ]:
x,y = valgen[0]

In [ ]:
ypred = model.predict_on_batch(x)

In [ ]:
i = 3
plt.figure(figsize=(20, 10))
plt.imshow(x[i, ...])
plt.imshow(y[i, ...].squeeze(), alpha=0.5)
plt.show()